# PySpark Logistic Regression

Our full dataset file has around 9 million samples. When trying to run feature_generator.

Machine:
* 2018 Mac Mini - 6 core

Docker Configuration:
* 9 CPUs
* 24 GB Ram
* 3 GB swap


# References:

* https://spark.apache.org/docs/2.2.0/mllib-feature-extraction.html
* https://towardsdatascience.com/countvectorizer-hashingtf-e66f169e2d4e
* https://medium.com/@dhiraj.p.rai/logistic-regression-in-spark-ml-8a95b5f5434c
* packaging spark job - https://developerzen.com/best-practices-writing-production-grade-pyspark-jobs-cb688ac4d20f



In [1]:
import sys
sys.path.append("../..")

import pyspark
from pyspark import SparkContext, SparkFiles
from pyspark.sql import SparkSession, DataFrameReader, SQLContext
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import NGram, CountVectorizer, VectorAssembler, Tokenizer, IDF
from pyspark.ml import Pipeline
from pyspark.sql.functions import when, lit, col
from pyspark.sql.types import NumericType


from sklearn.utils.class_weight import compute_class_weight

import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from pprint import pprint
import logging

import util.pyspark_util as pyu
import util.model_wrapper as mw



log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)
sns.set()
%matplotlib inline

# allow DEBUG to be set by command line
DEBUG = bool(os.environ.get("IPYNB_DEBUG", False))

N_CLASSES = 5
FEATURE_COLUMN = "review_body"

if DEBUG:
    MIN_DF = 1
    DF_PERCENTAGE = 0.001
    DATA_FILE = "/home/jupyter/dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-test1k-preprocessed.csv"
    TEST_STR="-test"
else:
    DF_PERCENTAGE = 0.001
    DATA_FILE = "/home/jupyter/dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-2m-preprocessed.csv"
    TEST_STR=""

REPORT_FILE = f"../../reports/201911-pyspark-report{TEST_STR}.csv"


spark = SparkSession.builder \
            .master("local[*]") \
            .appName("Pyspark Wrapper Test (local)") \
            .getOrCreate()




In [2]:



class Timer(object):
    
    def __init__(self, description: str):
        self.start_time = datetime.now()
        self.description = description
        
    def stop(self):
        self.end_time = datetime.now()
        self.print_duration_min()
        
    def print_duration_min(self):
        self.duration = int((self.end_time - self.start_time).total_seconds() / 60)
        print(f"{self.description} duration: {self.duration} minutes")
        
    def get_duraction_min(self):
        return self.duration






In [3]:
file_timer = Timer("file load time")
df = spark.read.csv(SparkFiles.get(DATA_FILE), 
                    header=True, 
                    inferSchema= True)
df.collect()
file_timer.stop()

file load time duration: 0 minutes


In [4]:
def build_ngrams(inputCol, min_df, n=3):
    log.info(f'Creating TFIDF using min_df: {min_df}')
    
    tokenizer = [Tokenizer(inputCol = inputCol, outputCol = "words")]
    
    ngrams = [
        NGram(n=i, inputCol="words", outputCol="{0}_grams".format(i))
        for i in range(1, n + 1)
    ]

    vectorizers = [
        CountVectorizer(minDF=min_df, inputCol="{0}_grams".format(i),
            outputCol="{0}_counts".format(i))
        for i in range(1, n + 1)
    ]

    assembler = [VectorAssembler(
        inputCols=["{0}_counts".format(i) for i in range(1, n + 1)],
        outputCol="raw_features"
    )]
    
    idf = [IDF().setInputCol("raw_features").setOutputCol("features")]
#     idf = [IDF(minDocFreq=min_df).setInputCol("raw_features").setOutputCol("features")]

    return Pipeline(stages=tokenizer + ngrams + vectorizers + assembler + idf)



pipeline_timer = Timer("pipeline time")
# calculate a reasonable min_df
min_df = max(int(df.count() * DF_PERCENTAGE), 1)

pipeline = build_ngrams(FEATURE_COLUMN, min_df)
df = pipeline.fit(df).transform(df)
pipeline_timer.stop()


INFO:__main__:Creating TFIDF using min_df: 2000


pipeline time duration: 3 minutes


In [5]:
df.printSchema()
pyu.show_df(df, ["star_rating", "features"], truncate=False)

root
 |-- _c0: integer (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- 1_grams: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 2_grams: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 3_grams: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 1_counts: vector (nullable = true)
 |-- 2_counts: vector (nullable = true)
 |-- 3_counts: vector (nullable = true)
 |-- raw_features: vector (nullable = true)
 |-- features: vector (nullable = true)

+-----------+------------------------------------------------------------------------------------------------------------------------------

# Split training and test data

In [6]:
split_timer = Timer("train test split")
train, test = df.randomSplit([0.9, 0.1], seed=1)
split_timer.stop()


train_size = train.count()
test_size = test.count()

print(f'Training size: {train_size} Test size: {test_size}')

train test split duration: 0 minutes
Training size: 1799736 Test size: 200264


# get number of features

In [7]:
type(train.select("features").limit(1).toPandas().features.values[0])
train.select("features").limit(1).toPandas().features.values[0].size

4068

# Assign class weights to handle imbalanced classes

In [8]:


# only do this for small files - takes too long for large datasets - we will custom compute this
# if DEBUG:
cw_timer = Timer("skelarn class weight")
labels = train.select("star_rating").toPandas().astype({"star_rating": np.int8})
class_weights = compute_class_weight('balanced', 
                                                  np.arange(1, N_CLASSES+1), 
                                                  labels.star_rating.tolist())
print(f'sklearn class weights: {class_weights}')
cw_timer.stop()
    


sklearn class weights: [1.42600786 3.0052951  2.21763898 1.19757655 0.37312794]
skelarn class weight duration: 2 minutes


In [9]:

train = train.withColumn("class_weights", lit(0))
train = train.withColumn("class_weights", when(train.star_rating == 1, class_weights[0]).otherwise(train.class_weights))
train = train.withColumn("class_weights", when(train.star_rating == 2, class_weights[1]).otherwise(train.class_weights))
train = train.withColumn("class_weights", when(train.star_rating == 3, class_weights[2]).otherwise(train.class_weights))
train = train.withColumn("class_weights", when(train.star_rating == 4, class_weights[3]).otherwise(train.class_weights))
train = train.withColumn("class_weights", when(train.star_rating == 5, class_weights[4]).otherwise(train.class_weights))


pyu.show_df(train, ["star_rating", "class_weights", "features"], 20, sample=True, truncate=True)


INFO:util.pyspark_util:sampling percentage: 1.1112740979788147e-05


+-----------+------------------+--------------------+
|star_rating|     class_weights|            features|
+-----------+------------------+--------------------+
|          2| 3.005295104825041|(4068,[0,4,6,8,15...|
|          5|0.3731279446445694|(4068,[4,8,10,14,...|
|          5|0.3731279446445694|(4068,[0,1,2,8,9,...|
|          5|0.3731279446445694|(4068,[3,6,35,63,...|
|          5|0.3731279446445694|(4068,[2,73,187,5...|
|          4| 1.197576548011565|(4068,[1,6,9,12,1...|
|          5|0.3731279446445694|(4068,[6,109,187,...|
|          4| 1.197576548011565|(4068,[0,8,21,22,...|
|          2| 3.005295104825041|(4068,[5,14,25,46...|
|          5|0.3731279446445694|(4068,[1,2,18,25,...|
|          1| 1.426007860040568|(4068,[5,6,7,9,11...|
|          5|0.3731279446445694|(4068,[13,16,33,9...|
|          4| 1.197576548011565|(4068,[8,10,2235]...|
|          5|0.3731279446445694|(4068,[168,184,26...|
|          1| 1.426007860040568|(4068,[0,5,7,10,1...|
+-----------+---------------

# Train our Model

In [10]:


import importlib
import util.constants as c
importlib.reload(pyu)
importlib.reload(mw)
importlib.reload(c)

feature_count = train.select("features").limit(1).toPandas().features.values[0].size

train_timer = Timer("traing time")
lr = LogisticRegression(labelCol="star_rating", 
                        featuresCol="features", 
                        weightCol="class_weights",
                        maxIter=100)


model = pyu.PysparkModel(model = lr,
                    train_df = train,
                    test_df = test,
                    label_column = "star_rating",
                    feature_column = "features",
                    n_classes = 5,
                         pipeline = pipeline,
                         file = DATA_FILE,
                         description=f'review_body-tfidf-df_none-ngram13-{df.count()}-{feature_count}-nolda-sampling_none{TEST_STR}',
                        model_dir="../../models")

report_dict, predict_test = model.run()
train_timer.stop()

pyu.show_df(predict_test, ["star_rating", "prediction", "rawPrediction", "probability"])



INFO:util.model_wrapper:derived name: LogisticRegression
INFO:util.model_wrapper:########################################
INFO:util.model_wrapper:Running model: name: LogisticRegression
	with file: /home/jupyter/dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-2m-preprocessed.csv
	with description: review_body-tfidf-df_none-ngram13-2000000-4068-nolda-sampling_none-LogisticRegression-star_rating
	status: new
INFO:util.model_wrapper:########################################
INFO:util.time_util:Start timer for: train_time_min
INFO:util.time_util:End timer for: train_time_min
INFO:util.time_util:Total time for train_time_min: 3.34
INFO:util.time_util:Start timer for: model_save_time_min
INFO:util.pyspark_util:Saving model to file: ../../models/review_body-tfidf-df_none-ngram13-2000000-4068-nolda-sampling_none-LogisticRegression-star_rating.pyspark
INFO:util.pyspark_util:Saving pipeline to file: ../../models/review_body-tfidf-df_none-ngram13-2000000-4068-nolda-sampling_none-LogisticRe

root
 |-- _c0: integer (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- 1_grams: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 2_grams: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 3_grams: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 1_counts: vector (nullable = true)
 |-- 2_counts: vector (nullable = true)
 |-- 3_counts: vector (nullable = true)
 |-- raw_features: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



INFO:util.time_util:End timer for: cr_time_min
INFO:util.time_util:Total time for cr_time_min: 5.26
INFO:util.model_wrapper:calculating confusion matrix...
INFO:util.time_util:Start timer for: cm_time_min
INFO:util.time_util:End timer for: cm_time_min
INFO:util.time_util:Total time for cm_time_min: 5.29


traing time duration: 18 minutes
+-----------+----------+--------------------+--------------------+
|star_rating|prediction|       rawPrediction|         probability|
+-----------+----------+--------------------+--------------------+
|          1|       3.0|[-10.661603880384...|[5.55111893109770...|
|          5|       4.0|[-10.663112482577...|[2.57492360776896...|
|          5|       5.0|[-10.662055028663...|[1.13905394122960...|
|          4|       4.0|[-10.664341097813...|[5.70024947144327...|
|          2|       3.0|[-10.662014368919...|[3.74789606254735...|
|          3|       2.0|[-10.664997203040...|[2.05828101026790...|
|          1|       2.0|[-10.665398748971...|[1.22581895934023...|
|          5|       4.0|[-10.664978840608...|[1.42020778224212...|
|          5|       5.0|[-10.663276426073...|[1.36115943611161...|
|          4|       4.0|[-10.663056714757...|[2.17940533120967...|
+-----------+----------+--------------------+--------------------+
only showing top 10 rows



In [11]:
predict_test.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- 1_grams: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 2_grams: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 3_grams: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- 1_counts: vector (nullable = true)
 |-- 2_counts: vector (nullable = true)
 |-- 3_counts: vector (nullable = true)
 |-- raw_features: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



# Evaluate our Model

Reference:
* https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html#multiclass-classification

In [12]:
report_dict

{'model_name': 'LogisticRegression',
 'description': 'review_body-tfidf-df_none-ngram13-2000000-4068-nolda-sampling_none-LogisticRegression-star_rating',
 'library': 'pyspark',
 'file': '/home/jupyter/dataset/amazon_reviews/amazon_reviews_us_Wireless_v1_00-2m-preprocessed.csv',
 'model_file': '../../models/review_body-tfidf-df_none-ngram13-2000000-4068-nolda-sampling_none-LogisticRegression-star_rating.pyspark',
 'pipeline_file': '../../models/review_body-tfidf-df_none-ngram13-2000000-4068-nolda-sampling_none-LogisticRegression-star_rating.pipeline',
 'status': 'success',
 'status_date': '2019-11-22 19:27:12',
 'classification_report': '{"1": {"precision": 0.7038692554826093, "recall": 0.7172443242473845, "f1-score": 0.7104938489195952, "support": 28102}, "2": {"precision": 0.30329631998139644, "recall": 0.3916372644696344, "f1-score": 0.34185177904462355, "support": 13321}, "3": {"precision": 0.3286404885081842, "recall": 0.4263864436619718, "f1-score": 0.371186359499976, "support": 1

In [13]:
pprint(json.loads(report_dict["classification_report"]))

{'1': {'f1-score': 0.7104938489195952,
       'precision': 0.7038692554826093,
       'recall': 0.7172443242473845,
       'support': 28102},
 '2': {'f1-score': 0.34185177904462355,
       'precision': 0.30329631998139644,
       'recall': 0.3916372644696344,
       'support': 13321},
 '3': {'f1-score': 0.371186359499976,
       'precision': 0.3286404885081842,
       'recall': 0.4263864436619718,
       'support': 18176},
 '4': {'f1-score': 0.42217745967799536,
       'precision': 0.4030770909487697,
       'recall': 0.4431780690441198,
       'support': 33341},
 '5': {'f1-score': 0.805782314613098,
       'precision': 0.861976705914829,
       'recall': 0.7564664008050389,
       'support': 107324},
 'accuracy': 0.6445791555147206,
 'macro avg': {'f1-score': 0.5302983523510576,
               'precision': 0.5201719721671577,
               'recall': 0.5469825004456299,
               'support': 200264},
 'weighted avg': {'f1-score': 0.658243063046397,
                  'precision': 0

In [14]:
print(np.array(json.loads(report_dict["confusion_matrix"])))

[[20156  5132  1794   428   592]
 [ 4081  5217  2951   646   426]
 [ 1833  3682  7750  3390  1521]
 [  788  1419  5897 14776 10461]
 [ 1778  1751  5190 17418 81187]]


In [15]:
import json

def calculate_score(cr: dict):
    
    values = []
    values.append(cr["1"]["recall"])
    values.append(cr["2"]["recall"])
    values.append(cr["3"]["recall"])
    values.append(cr["4"]["recall"])
    values.append(cr["5"]["precision"])
    
    mean = 0
    for v in values:
        if v == 0:
            mean = 0
            break
        else:
            mean += 1 / v
    if mean > 0:
        mean = len(values) / mean

    return mean




score = calculate_score(json.loads(report_dict['classification_report']))
print(f'Overall score: {score}')

Overall score: 0.514962119307262


# Save Report

In [16]:
if os.path.exists(REPORT_FILE):
    report_df = pd.read_csv(REPORT_FILE, quotechar="'")
else:
    report_df = pd.DataFrame()
report_df = report_df.append(report_dict, ignore_index=True)
report_df.to_csv(REPORT_FILE, index=False, quotechar="'")
report_df

,classification_report,cm_time_min,confusion_matrix,cr_time_min,description,file,library,model_file,model_name,model_save_time_min,pipeline_file,predict_time_min,status,status_date,test_examples,test_features,total_time_min,train_examples,train_features,train_time_min
0,152,526.00,1885]]',0.35,review_body-tfidf-df_none-ngram13-49784-4254-n...,/home/jupyter/dataset/amazon_reviews/amazon_re...,pyspark,../../models/review_body-tfidf-df_none-ngram13...,LogisticRegression,0.06,../../models/review_body-tfidf-df_none-ngram13...,0.0,success,2019-11-22 07:50:22,5058.0,4254.0,1.00,44726.0,4254.0,0.30
1,"{""1"": {""precision"": 0.6406469760900141, ""recal...",0.32,"[[911, 280, 132, 46, 36], [221, 214, 150, 61, ...",0.45,review_body-tfidf-df_none-ngram13-99567-4159-n...,/home/jupyter/dataset/amazon_reviews/amazon_re...,pyspark,../../models/review_body-tfidf-df_none-ngram13...,LogisticRegression,0.05,../../models/review_body-tfidf-df_none-ngram13...,0.0,success,2019-11-22 07:53:26,10023.0,4159.0,1.15,89544.0,4159.0,0.33
2,"{""1"": {""precision"": 0.6127167630057804, ""recal...",0.28,"[[424, 173, 80, 31, 27], [90, 97, 73, 35, 19],...",0.30,review_body-tfidf-df_none-ngram13-49784-4254-n...,/home/jupyter/dataset/amazon_reviews/amazon_re...,pyspark,../../models/review_body-tfidf-df_none-ngram13...,LogisticRegression,0.04,../../models/review_body-tfidf-df_none-ngram13...,0.0,success,2019-11-22 18:23:18,5058.0,4254.0,0.89,44726.0,4254.0,0.27
3,"{""1"": {""precision"": 0.6406469760900141, ""recal...",0.37,"[[911, 280, 132, 46, 36], [221, 214, 150, 61, ...",0.41,review_body-tfidf-df_none-ngram13-99567-4159-n...,/home/jupyter/dataset/amazon_reviews/amazon_re...,pyspark,../../models/review_body-tfidf-df_none-ngram13...,LogisticRegression,0.04,../../models/review_body-tfidf-df_none-ngram13...,0.0,success,2019-11-22 18:26:20,10023.0,4159.0,1.14,89544.0,4159.0,0.32
4,"{""1"": {""precision"": 0.6762513312034079, ""recal...",0.55,"[[1905, 566, 257, 48, 64], [408, 465, 326, 82,...",0.59,review_body-tfidf-df_none-ngram13-199134-4082-...,/home/jupyter/dataset/amazon_reviews/amazon_re...,pyspark,../../models/review_body-tfidf-df_none-ngram13...,LogisticRegression,0.04,../../models/review_body-tfidf-df_none-ngram13...,0.0,success,2019-11-22 18:30:24,19966.0,4082.0,1.70,179168.0,4082.0,0.52
5,"{""1"": {""precision"": 0.6981843575418994, ""recal...",1.36,"[[4999, 1352, 474, 114, 142], [1023, 1170, 774...",1.41,review_body-tfidf-df_none-ngram13-497835-4107-...,/home/jupyter/dataset/amazon_reviews/amazon_re...,pyspark,../../models/review_body-tfidf-df_none-ngram13...,LogisticRegression,0.04,../../models/review_body-tfidf-df_none-ngram13...,0.0,success,2019-11-22 18:38:59,49846.0,4107.0,3.86,447989.0,4107.0,1.05
6,"{""1"": {""precision"": 0.7026024905554779, ""recal...",2.63,"[[10043, 2635, 928, 224, 272], [1991, 2504, 13...",2.60,review_body-tfidf-df_none-ngram13-995688-4073-...,/home/jupyter/dataset/amazon_reviews/amazon_re...,pyspark,../../models/review_body-tfidf-df_none-ngram13...,LogisticRegression,0.04,../../models/review_body-tfidf-df_none-ngram13...,0.0,success,2019-11-22 18:55:13,99684.0,4073.0,7.22,896004.0,4073.0,1.95
7,"{""1"": {""precision"": 0.7038692554826093, ""recal...",5.29,"[[20156, 5132, 1794, 428, 592], [4081, 5217, 2...",5.26,review_body-tfidf-df_none-ngram13-2000000-4068...,/home/jupyter/dataset/amazon_reviews/amazon_re...,pyspark,../../models/review_body-tfidf-df_none-ngram13...,LogisticRegression,0.04,../../models/review_body-tfidf-df_none-ngram13...,0.0,success,2019-11-22 19:27:12,200264.0,4068.0,13.93,1799736.0,4068.0,3.34
